In [5]:
# import libraries 
# ----------------
%matplotlib notebook
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import matplotlib.pyplot as plt

In [6]:
# check .mat file 
# ---------------
from scipy.io import whosmat
whosmat('OBC_synthetic_data_PS.mat') # to list variables inside mat file

[('vx', (60000, 2400), 'double'),
 ('vz', (60000, 2400), 'double'),
 ('pressure', (60000, 2400), 'double')]

In [7]:
# load .mat file 
# --------------
from scipy.io import loadmat
mat = loadmat('OBC_synthetic_data_PS.mat')  # to load mat file
# convert them to numpy array
vx = mat['vx']
vz = mat['vz']
pressure = mat['pressure']

In [8]:
# initialize parameters 
# ---------------------
nx = vx.shape[1] # number of cells in x direction
dx = 0.25        # cell size
dt = 5e-6        # time step
alpha = 1800     # P-wave velocity
beta = 700       # S-wave velocity
rho = 1600       # density

In [ ]:
# calculate wavefield gradients 
# -----------------------------
from sympy.calculus import finite_diff_weights # see http://docs.sympy.org/latest/modules/calculus/index.html
c = finite_diff_weights(1, [-3/2, -1/2, 1/2, 3/2], 0) # FD coefficients according to Fornberg (1988)
c = c[-1][-1] # FD weights for 1st derivative

# allocate zero gradients
gradx_vx       = np.zeros((      vx.shape[0],nx))
gradx_vz       = np.zeros((      vz.shape[0],nx))
gradx_pressure = np.zeros((pressure.shape[0],nx))

for n in range(2,(vx.shape[1]-2)):
    # horizontal gradients of vx
    gradx_vx[:,n] = ((c[0]*vx[:,n-1]) + (c[1]*vx[:,n])   + (c[2]*vx[:,n+1]) + (c[3]*vx[:,n+2])) / dx
    # horizontal gradients of vz
    gradx_vz[:,n] = ((c[0]*vz[:,n-2]) + (c[1]*vz[:,n-1]) + (c[2]*vz[:,n])   + (c[3]*vz[:,n+1])) / dx
    # horizontal gradient of pressure
    gradx_pressure[:,n] = ((c[0]*pressure[:,n-2]) + (c[1]*pressure[:,n-1]) \
                         + (c[2]*pressure[:,n])   + (c[3]*pressure[:,n+1])) / dx
    
# P/S separation 
# --------------
# upgoing P-wave on vz
vz_P_up = (1/2.) * (vz + ((beta*((2*beta)/alpha)) * (gradx_vx.cumsum(axis=0)*dt)) - ((1/(alpha * rho)) * pressure))
# upgoing S-wave on vx
vx_S_up = (1/2.) * (vx - ((2*beta) * (gradx_vz.cumsum(axis=0)*dt)) + ((1/rho) * (gradx_pressure.cumsum(axis=0)*dt)))

In [ ]:
# plotting configuration 
# ----------------------
figure()

# recorded vz
subplot(2,2,1)
set(gcf, 'color', 'w');
set (gca, 'Fontsize', 20);
imagesc(vz); caxis([-1 1]/3e1)
xlabel('Incidence angle ({\circ})', 'Fontsize',20);
set(gca,'XTick', [1200 1235 1271 1307 1346 1387 1431 1480 1536 1600 1677])
set(gca,'XTickLabel',{'0', '','','','' '25','','','','', '50'});
ylabel('Time (s)', 'Fontsize',20)
set(gca,'YTick', [1 10000 20000 30000 40000 50000 59000]);
set(gca,'YTickLabel',{'0.00', '', '0.10', '', '0.20', '', '0.30'});
title('Recorded v_z', 'Fontsize',20)
axis([1200 1677 1 59000])
colormap('cs_bluewhitered')

# P/S separated upgoing vz
subplot(2,2,3)
set(gcf, 'color', 'w');
set (gca, 'Fontsize', 20);
imagesc(vz_P_up); caxis([-1 1]/3e1)
xlabel('Incidence angle ({\circ})', 'Fontsize',20);
set(gca,'XTick', [1200 1235 1271 1307 1346 1387 1431 1480 1536 1600 1677])
set(gca,'XTickLabel',{'0', '','','','' '25','','','','', '50'});
ylabel('Time (s)', 'Fontsize',20)
set(gca,'YTick', [1 10000 20000 30000 40000 50000 59000]);
set(gca,'YTickLabel',{'0.00', '', '0.10', '', '0.20', '', '0.30'});
title('Upgoing P-wave on v_z - filters', 'Fontsize',20)
axis([1200 1677 1 59000])
colormap('cs_bluewhitered')

# recorded vx
subplot(2,2,2)
set(gcf, 'color', 'w');
set (gca, 'Fontsize', 20);
imagesc(vx); caxis([-1 1]/3e1)
xlabel('Incidence angle ({\circ})', 'Fontsize',20);
set(gca,'XTick', [1200 1235 1271 1307 1346 1387 1431 1480 1536 1600 1677])
set(gca,'XTickLabel',{'0', '','','','' '25','','','','', '50'});
ylabel('Time (s)', 'Fontsize',20)
set(gca,'YTick', [1 10000 20000 30000 40000 50000 59000]);
set(gca,'YTickLabel',{'0.00', '', '0.10', '', '0.20', '', '0.30'});
title('Recorded v_x', 'Fontsize',20)
axis([1200 1677 1 59000])
colormap('cs_bluewhitered')

# P/S separated upgoing vx
subplot(2,2,4)
set(gcf, 'color', 'w');
set (gca, 'Fontsize', 20);
imagesc(vx_S_up); caxis([-1 1]/3e1)
xlabel('Incidence angle ({\circ})', 'Fontsize',20);
set(gca,'XTick', [1200 1235 1271 1307 1346 1387 1431 1480 1536 1600 1677])
set(gca,'XTickLabel',{'0', '','','','' '25','','','','', '50'});
ylabel('Time (s)', 'Fontsize',20)
set(gca,'YTick', [1 10000 20000 30000 40000 50000 59000]);
set(gca,'YTickLabel',{'0.00', '', '0.10', '', '0.20', '', '0.30'});
title('Upgoing S-wave on v_x - filters', 'Fontsize',20)
axis([1200 1677 1 59000])
colormap('cs_bluewhitered')

plt.show()